In [1]:
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
import numpy as np
from statistics import mean
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
#df = pd.read_csv('./data/askwomen.csv', low_memory=False)
#df['post_score'].describe()[6]
#df.sort_values(by='post_score', ascending=False).head(10)

In [3]:
#df['post_score'].head(20)

In [4]:
#df[['post_score']] = scaler.fit_transform(df[['post_score']])
#df['post_score'].head(20)

In [5]:
path_to_csv = './data/'
csvs = [pos_csv for pos_csv in os.listdir(path_to_csv) if pos_csv.endswith('.csv')]
csvs = ['askwomen.csv']
csv_dict = dict()
for page in csvs:
    #url = "https://raw.githubusercontent.com/rer145/cis572-project/master/data/"+str(page)+".csv"
    #raw_data = requests.get(url).content
    #df = pd.read_csv(io.StringIO(raw_data.decode('utf-8')))
    df = pd.read_csv(path_to_csv + page, low_memory=False)
    csv_dict[page] = df

In [6]:
# remove columns that aren't predictive
for key in csv_dict.keys():
    csv_dict[key].drop('post_gilded', axis=1, inplace=True)
    csv_dict[key].drop('post_gilded_silver', axis=1, inplace=True)
    csv_dict[key].drop('post_gilded_gold', axis=1, inplace=True)
    csv_dict[key].drop('post_gilded_platinum', axis=1, inplace=True)
    csv_dict[key].drop('post_likes', axis=1, inplace=True)
    csv_dict[key].drop('post_num_comments', axis=1, inplace=True)
    csv_dict[key].drop('post_num_crossposts', axis=1, inplace=True)
    csv_dict[key].drop('post_num_reports', axis=1, inplace=True)
    csv_dict[key].drop('post_ups', axis=1, inplace=True)
    csv_dict[key].drop('post_downs', axis=1, inplace=True)

In [7]:
#Remove columns that only have a singular value
for key in csv_dict.keys():
    unique = [c for c in csv_dict[key].columns if len(set(csv_dict[key][c])) == 1]
    print(key)
    print(unique)
    print()
    csv_dict[key].drop(unique, axis=1, inplace=True)

askwomen.csv
['post_archived', 'post_is_original_content', 'post_is_video', 'post_pinned', 'subreddit', 'day_of_week']



In [8]:
for key in csv_dict.keys():
    print('parsing', key)
    #Change post_distinguished to numerical values, only None and Moderator values
    if 'post_distinguished' in csv_dict[key]:
        csv_dict[key]['post_distinguished'] = csv_dict[key]['post_distinguished'].map({None: 0, 'moderator': 1})
        
    #Mark edited posts with a 1
    if 'post_edited' in csv_dict[key]:
        csv_dict[key]['post_edited'] = (csv_dict[key]['post_edited'] == 'False')*1
        
    #Make post_over_18 binary
    if 'post_over_18' in csv_dict[key]:
        csv_dict[key]['post_over_18'] = csv_dict[key]['post_over_18'].astype('category').cat.codes
        
    #csv_dict[key] = csv_dict[key].dropna(csv_dict[key].median())

print('done')

parsing askwomen.csv
done


In [9]:
#for key in csv_dict.keys():
#    csv_dict[key] = csv_dict[key].sort_values('post_score', ascending=False)
#    cutoff_pos = round(csv_dict[key].shape[0]*.25)
#    cutoff_score = csv_dict[key].iloc[cutoff_pos]['post_score']
#    print('Subreddit:',key,'\t Number of popular posts:',cutoff_pos,'\t Popular score cutoff:',int(cutoff_score))
    
#    csv_dict[key].loc[csv_dict[key]['post_score'] >= cutoff_score, 'popular'] = 1
#    csv_dict[key].loc[csv_dict[key]['post_score'] < cutoff_score, 'popular'] = 0

In [10]:
#def is_popular(s):
#    return s >= min_popular_score###

#for key in csv_dict.keys():
#    df = csv_dict[key]
#    df[['post_score']] = scaler.fit_transform(df[['post_score']])
#    
###    max_post_score = max(df['post_score'])
#    min_popular_score = max_post_score * .25#

    #print(key)
    #print('Highest Scoring Post', max_post_score)
    #print('Min Post Score (Top 25%)', min_popular_score)
    #print()

    #df['popular'] = df['post_score'].apply(is_popular)
    #df.loc[df['post_score'] >= min_popular_score -100]
    
    #csv_dict[key] = df

In [14]:
for key in csv_dict.keys():
    cutoff_score = csv_dict[key]['post_score'].describe()[6]
    csv_dict[key].loc[csv_dict[key]['post_score'] >= cutoff_score, 'popular'] = 1
    csv_dict[key].loc[csv_dict[key]['post_score'] < cutoff_score, 'popular'] = 0

In [15]:
def prepare_data(data):
    
    print(data.groupby(by='popular')['post_title_words'].count())
    
    
    #remove cols
    if 'post_score' in data.columns:
        data.drop('post_score', axis=1, inplace=True)
    
    #split into test and training
    X = data.drop('popular', axis=1)
    y = data['popular']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    #print('First split of data')
    #print('X_train', X_train.shape)
    #print('X_test', X_test.shape)
    #print('y_train', y_train.shape)
    #print('y_test', y_test.shape)
    
    #split into validation and training
    #X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)
    
    #print('Second split of data')
    #print('X_train', X_train.shape)
    #print('X_val', X_val.shape)
    #print('y_train', y_train.shape)
    #print('y_val', y_val.shape)
    
    #scale the data
    X_train = scaler.fit_transform(X_train)
    #X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)
    
    #print('Data sizes')
    #print('X_train', X_train.shape)
    #print('X_val', X_val.shape)
    #print('X_test', X_test.shape)
    #print('y_train', y_train.shape)
    #print('y_val', y_val.shape)
    #print('y_test', y_test.shape)
    
    #return data
    #return X_train, X_val, X_test, y_train, y_val, y_test
    return X_train, X_test, y_train, y_test

In [16]:
for key in csv_dict.keys():
    print(key)
    X_train, X_test, y_train, y_test = prepare_data(csv_dict[key])
    print()

askwomen.csv
popular
0.0    740
1.0    251
Name: post_title_words, dtype: int64



In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier()
gnb = GaussianNB()
knn = KNeighborsClassifier()
svc = SVC()

classifiers = [dtc, rfc, gnb, knn, svc]

In [18]:
names = ['Decision Tree', 'Random Forest', 'Gaussian Naive Bayes', 'K-Nearest Neighbors', 'Support Vector Classifer']
cols = ['# Instances', '# Features'] + names + ['Hard Voting', 'Soft Voting', 'Bagging']
val_results = pd.DataFrame(index=csv_dict.keys(), columns=cols)
val_results.head()

,# Instances,# Features,Decision Tree,Random Forest,Gaussian Naive Bayes,K-Nearest Neighbors,Support Vector Classifer,Hard Voting,Soft Voting,Bagging
askwomen.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
from sklearn.metrics import confusion_matrix
#accuracy_scores = []

for key in csv_dict.keys():
    #X_train, X_val, X_test, y_train, y_val, y_test = prepare_data(csv_dict[key])
    X_train, X_test, y_train, y_test = prepare_data(csv_dict[key])
    print('Subreddit:', key)
    #print()
    #accuracy_scores = []
    for name, algo in zip(names, classifiers):
    #for algo in classifiers:
        #if sum([1 for x in list(y_val) if x is True]) > 0 and sum([1 for x in list(y_val) if x is False]) > 0:
        #if len(set(y_val.values)) == 2:
        if len(set(y_test.values)) == 2:
            algo.fit(X_train, y_train)
            #pred = algo.predict(X_val)
            pred = algo.predict(X_test)

            val_results.loc[key]['# Instances'] = X_train.shape[0]
            val_results.loc[key]['# Features'] = X_train.shape[1]
            #val_results.loc[key][name] = round(accuracy_score(y_val, pred), 3)
            val_results.loc[key][name] = round(accuracy_score(y_test, pred), 3)
            
            
            print(name)
            tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
            print('tn, fp, fn, tp')
            print(tn, fp, fn, tp)

        #print(name, '\n')
        #print('Accuracy score:', accuracy_score(y_val, pred))
        #accuracy_scores.append(round(accuracy_score(y_val, pred),3))
        #print('F1 score:', f1_score(y_val, pred))
        #print('Precision score:', precision_score(y_val, pred))
        #print('Recall score:', recall_score(y_val, pred))
        #print('ROC/AUC score:', roc_auc_score(y_val, pred))
        #print()
    #print()
    #print('Min score:', min(accuracy_scores))
    #print('Mean score:', mean(accuracy_scores))
    #print('Max score:',max(accuracy_scores))
    #print()
    #print('-'*60)
print('done')

popular
0.0    740
1.0    251
Name: post_title_words, dtype: int64
Subreddit: askwomen.csv
Decision Tree
tn, fp, fn, tp
112 37 38 12
Random Forest
tn, fp, fn, tp
149 0 50 0
Gaussian Naive Bayes
tn, fp, fn, tp
29 120 9 41
K-Nearest Neighbors
tn, fp, fn, tp
149 0 50 0
Support Vector Classifer
tn, fp, fn, tp
149 0 50 0
done


In [21]:
val_results

,# Instances,# Features,Decision Tree,Random Forest,Gaussian Naive Bayes,K-Nearest Neighbors,Support Vector Classifer,Hard Voting,Soft Voting,Bagging
askwomen.csv,792,10868,0.648,0.749,0.352,0.749,0.749,NaN,NaN,NaN


tn, fp, fn, tp
149 0 50 0
